In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import custom_fn as cf
import sklearn.svm
import matplotlib.pyplot as plt
#import sklearn.lda # Poor Performance
#import sklearn.decomposition #For PCA. Poor Performance

In [ ]:
# Load Data
X_add, y_add = cf.loadData_and_addFeatures()

In [ ]:
# Visualize Data
#cf.saveAllPlots(X_add, y_add) #Save all plots
print(X_add.columns)
#cf.plotFeature(X_add, y_add,'LWR(blinkStrength)')

In [ ]:
#Select Features to Run
features=[]
features.append('N(blinks)_1m')
features.append('N(50<blinkStrength)_1m')
features.append('N(eeg<-500)_1m')
features.append('N(alphaLow>1.677e+007)_1m')
features.append('N(betaLow>1.674e+007)_1m')
features.append('N(1e+005<theta<5e+005)_1m')
features.append('blinkStrength') #For deep sleep detection
features.append('N(1e+005<alphaHigh<2e+005)_1m')

X_select = X_add[features].values
print(X_select.shape)

In [ ]:
# Shuffle Data
X_shuffled, y_shuffled = cf.shuffleData(X_select, y_add)
print(X_shuffled.shape, y_shuffled.shape)

In [ ]:
#Divide Examples
idx_train = int(X_shuffled.shape[0] * 0.7)
idx_val = int(X_shuffled.shape[0] * 0.9)
print(idx_train, idx_val - idx_train, X_shuffled.shape[0] - idx_val)

In [ ]:
# Divide examples using Mini-batch
X_train = X_shuffled[0:idx_train]
y_train = y_shuffled[0:idx_train]
X_val = X_shuffled[idx_train + 1:idx_val]
y_val = y_shuffled[idx_train + 1:idx_val]
X_test = X_shuffled[idx_val + 1:]
y_test = y_shuffled[idx_val + 1:]

In [ ]:
# Test with default options
svc = sklearn.svm.SVC(kernel='rbf', C=8, gamma=0.1).fit(X_train, y_train)
print(svc)

In [ ]:
# Construct Accuracy metrics
# Simple Accuracy
pred_pos = svc.predict(X_val)
simple_accuracy = np.mean(pred_pos == y_val)
# F1 score
Scores_sleepy = cf.F1_score(pred_pos, y_val, class_idx=1)
Scores_deep = cf.F1_score(pred_pos, y_val, class_idx=2)
# Print Accuracy metrics
print(simple_accuracy)
print(Scores_sleepy)
print(Scores_deep)
print(svc.n_support_)

In [ ]:
# Parameter tuning
C_values = [2 ** (x) for x in range(-11, 4)]
gamma_values = [10 ** x for x in range(-3, 2)]
best_svc = svc;
best_scores = [accuracy, F1, P, R]
for c in C_values:
    for g in gamma_values:
        svc_dummy = sklearn.svm.SVC(kernel='rbf', C=c, gamma=g).fit(X_train, y_train)
        dummy_pred_pos = svc_dummy.predict(X_val)
        dummy_simple_accuracy = np.mean(pred_pos == y_val)
        dummy_Scores = F1_score(dummy_pred_pos, y_val)
        if dummy_F1 > F1:
            best_svc = svc_dummy
            best_scores = (dummy_simple_accuracy, dummy_Scores)
print(best_svc)
print(best_scores)
print(best_svc.n_support_)

In [ ]:
# Classify the test set
test_pred_pos = svc.predict(X_test)
test_simple_accuracy = np.mean(test_pred_pos == y_test)
test_F1_sleepy = cf.F1_score(test_pred_pos, y_test, class_idx=1)
test_F1_deep = cf.F1_score(test_pred_pos, y_test, class_idx=2)
print(test_simple_accuracy)
print(test_F1_sleepy)
print(test_F1_deep)

In [ ]:
# Code for transmitting test results over the network
#import serial as s
#ser=s.Serial('COM7',9600)
# Loop

import time, threading
def repeated_cycle():
    # Input raw csv files
    filename = 'DATA(150827).csv'
    # Add new features
    X_new_add = cf.loadNewData_and_addFeatures(filename)
    if X_new_add.iloc[0].values[0]!='Invalid values':
        X_this = X_new_add[features].values
        # Classify
        state = svc.predict(X_this[-1])
        print(int(state[0]))
        # Output classification results
        #ser.write(int(state[0])

            # Code for time adjustment
        threading.Timer(1,repeated_cycle).start()
repeated_cycle()